In [ ]:
# Standard Imports
import numpy as np
import matplotlib.pyplot as plt
import csv, datetime
import matplotlib.dates as mdates

def MakeDataFromTxt(filepath):
    """
    Helper function to parse text files in IFM proprietary format (tags.txt or tags.csv)
    """
    with open(filepath, 'r') as file:
        reader = csv.reader(file)

        db = dict()                          # Dictionary that will hold k,v pairs of 
        recent_EPC = ''                      # The most recent EPC that was read, and data is being assigned to
        Time_Or_Data = True                  # Time is true, Data is false
        reached_24 = False

        for l in reader:                     # Iterate over entire text file
            line = l[0]                      # Each line only has one token, so we select it
            
            if len(line)==24:                # If it is an EPC
                db[line] = [[],[]]           # Create new k,v pair in database
                recent_EPC = line            # Current EPC to add to is selected
            
            elif line == '[':                # If start bracket, beginning to collect an array of data
                data = []

            elif line == ']':                # End bracket means end of data array
                if len(data)!=0:
                    if Time_Or_Data:
                        db[recent_EPC][0] = data
                        Time_Or_Data = False
                    else:
                        db[recent_EPC][1] = data
                        Time_Or_Data = True
                data = []

            elif len(line)==8:               # If it is a time stamp
                h,m,s = line.split(':')      # Parse timestamp into hours, minutes, seconds

                if int(h)==23:
                    reached_24 = True        # If end of day reached, must move to Day 2

                if reached_24 and int(h) < 12:
                    day=2
                else:
                    day=1

                data.append(datetime.datetime(1970, 2, day, int(h), int(m), int(s)))
            
            elif line ==' ':               # If blank space, then we skip to parse next line
                pass

            else:                          # If it is a data point
                try:
                    data.append(float(line))
                except:
                    pass                   # Error catching case just in case, so annoying errors don't crash program
    return db

In [ ]:
r1 = MakeDataFromTxt('RSSI_62972.csv') # Put path to RSSI data here
t1 = MakeDataFromTxt('tags_62972.csv') # Put path to Temp data here


# Combined will be the dictionary containing Temperature Data
combined = dict()
for d in [ t1, ]:
    for k,v in d.items():
        if k in combined:                                 # if tag is already in dict
            for i in range(len(v[0])):                    # iterate over all time points
                if v[0][i] in combined[k]:                # if time point is already in dict
                    combined[k][v[0][i]].append(v[1][i])
                else:                                     # if time point is not in dict
                    combined[k][v[0][i]] = [v[1][i]]
        else:             # if tag is not in dict
            combined[k] = dict()
            for i in range(len(v[0])):
                combined[k][v[0][i]] = [v[1][i]]
for k,v in combined.items():
    for k2,v2 in v.items():
        combined[k][k2] = sum(v2)/len(v2) # avg all points taken during the same second

# Rcombined will be the dictionary containing RSSI Data
rcombined = dict()
for d in [ r1, ]:
    for k,v in d.items():
        if k in rcombined:                                 # if tag is already in dict
            for i in range(len(v[0])):                    # iterate over all time points
                if v[0][i] in rcombined[k]:                # if time point is already in dict
                    rcombined[k][v[0][i]].append(v[1][i])
                else:                                     # if time point is not in dict
                    rcombined[k][v[0][i]] = [v[1][i]]
        else:             # if tag is not in dict
            rcombined[k] = dict()
            for i in range(len(v[0])):
                rcombined[k][v[0][i]] = [v[1][i]]
for k,v in rcombined.items():
    for k2,v2 in v.items():
        rcombined[k][k2] = sum(v2)/len(v2) # avg all points taken during the same second

In [ ]:
# List of the EPC endings we want to graph
tags = [
    "460E",
    "710D",
    "135C",
    "B642",
    "8320",
    "2C68",
    "7566",
    "9D1F",
    "152D",
    "88A5",
    "2B1C",
    "4A59",
    "897D",
    "B0A5",
    "87B4",
    "0F4B",
    "44CD",
    "362B",
    "3C35",
    "828F",
]

In [ ]:
# Iterate over each tag and make a plot with Temperature on 
# top and RSSI on Bottom, on the same Time Scale (shared X-axis)

for tag in tags:

    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(10,10)) # Stacked plot
    ax1.set_title("Tag:" + tag)
    ax1.set_ylabel("Temp (C)")
    ax2.set_ylabel("RSSI (dBm)")

    for k,v in combined.items(): # Plot Temperature
        if k[-4:]==tag:
            s = sorted( v.items() )
            time, data = [ j[0] for j in s ], [ j[1] for j in s ]
            ax1.plot(time, data)
    
    for k,v in rcombined.items():      # Plot RSSI
        if k[-4:]==tag:                # Chop EPC to the last four characters in string
            s = sorted( v.items() )
            time, data = [ j[0] for j in s ], [ j[1] for j in s ]
            ax2.plot(time, data)
    
    fig.savefig("Tag_" + tag + ".png")